<a href="https://colab.research.google.com/github/lahari2005-sketch/Helmet-Detection-project/blob/main/helmet_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# HELMET DETECTION - WITH CUSTOM TRAINED MODEL
# ============================================================================

# STEP 1: Install YOLOv5
# ============================================================================
print("⏳ STEP 1/6: Installing YOLOv5... (2-3 minutes)")

import os
os.system('git clone https://github.com/ultralytics/yolov5 2>&1 | grep -v "already exists"')
os.chdir('yolov5')
os.system('pip install -qr requirements.txt')

print("✅ YOLOv5 installed!\n")

# STEP 2: Import Required Libraries
# ============================================================================
print("⏳ STEP 2/6: Loading libraries...")

import torch
import cv2
from IPython.display import Image as IPImage, display, HTML
from google.colab.patches import cv2_imshow
from google.colab import files
import numpy as np
import urllib.request

print("✅ Libraries loaded!\n")

# STEP 3: Download Pre-trained Helmet Detection Model
# ============================================================================
print("⏳ STEP 3/6: Downloading CUSTOM helmet detection model...")
print("📥 This model is specifically trained to detect helmets!\n")

# Method 1: Try to download from GitHub
model_urls = [
    # Pre-trained helmet detection models
    "https://github.com/hLahari/helmet-detection/raw/main/best.pt",
    "https://huggingface.co/keremberke/yolov5m-helmet/resolve/main/best.pt",
]

model_downloaded = False

# Try each URL
for idx, url in enumerate(model_urls):
    try:
        print(f"   Trying source {idx+1}...")
        urllib.request.urlretrieve(url, "helmet_model.pt")
        model_downloaded = True
        print(f"   ✅ Model downloaded successfully from source {idx+1}!")
        break
    except:
        print(f"   ⚠️  Source {idx+1} unavailable, trying next...")
        continue

# If download fails, use alternative method
if not model_downloaded:
    print("\n📦 Automatic download failed. Using alternative method...")
    print("🔄 Installing from Roboflow Universe...")

    os.system('pip install -q roboflow')

    from roboflow import Roboflow

    try:
        # Download from Roboflow public dataset
        rf = Roboflow(api_key="placeholder")
        project = rf.workspace("roboflow-universe-projects").project("helmet-detection-3fxl1")
        dataset = project.version(1).download("yolov5")

        # Copy the trained model
        os.system('cp helmet-detection-3fxl1-1/train/weights/best.pt helmet_model.pt')
        model_downloaded = True
        print("✅ Model downloaded from Roboflow!")
    except:
        print("⚠️  Roboflow method failed too.")

# Final fallback: Manual upload
if not model_downloaded:
    print("\n" + "="*70)
    print("📤 MANUAL UPLOAD REQUIRED")
    print("="*70)
    print("Please download a helmet detection model manually:")
    print("\n1. Visit: https://universe.roboflow.com/search?q=helmet+detection")
    print("2. Choose a 'YOLOv5' format model")
    print("3. Download the 'best.pt' file")
    print("4. Upload it below when prompted\n")
    print("="*70)

    uploaded = files.upload()

    for filename in uploaded.keys():
        os.rename(filename, 'helmet_model.pt')
        model_downloaded = True
        print(f"✅ Model '{filename}' uploaded successfully!")
        break

print("\n")

# STEP 4: Load the Helmet Detection Model
# ============================================================================
print("⏳ STEP 4/6: Loading helmet detection model...")

try:
    # Load custom helmet model
    model = torch.hub.load('ultralytics/yolov5', 'custom',
                          path='helmet_model.pt',
                          force_reload=True)
    model.conf = 0.45  # Confidence threshold (45%)
    model.iou = 0.45   # IoU threshold

    print("✅ Custom helmet model loaded successfully!")
    print(f"📊 Model can detect: {model.names}")

except Exception as e:
    print(f"⚠️  Error loading custom model: {e}")
    print("🔄 Falling back to general YOLOv5 model...")
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    model.conf = 0.45

print("\n")

# STEP 5: Helper Functions for Detection
# ============================================================================
print("⏳ STEP 5/6: Setting up detection functions...")

def draw_custom_boxes(img, results):
    """Draw colored boxes based on helmet detection"""

    detections = results.pandas().xyxy[0]

    for idx, row in detections.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        conf = row['confidence']
        class_name = row['name']

        # Color coding
        if 'helmet' in class_name.lower() or 'with' in class_name.lower():
            color = (0, 255, 0)  # GREEN for helmet
            label = f"WITH HELMET {conf:.2f}"
        elif 'without' in class_name.lower() or 'no' in class_name.lower():
            color = (0, 0, 255)  # RED for no helmet
            label = f"NO HELMET {conf:.2f}"
        else:
            color = (255, 0, 0)  # BLUE for others
            label = f"{class_name} {conf:.2f}"

        # Draw rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)

        # Draw label background
        cv2.rectangle(img, (x1, y1-30), (x1+len(label)*10, y1), color, -1)

        # Draw label text
        cv2.putText(img, label, (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return img

def analyze_results(results):
    """Analyze detection results and provide summary"""
    detections = results.pandas().xyxy[0]

    helmet_count = 0
    no_helmet_count = 0
    person_count = 0
    bike_count = 0

    for idx, row in detections.iterrows():
        class_name = row['name'].lower()

        if 'helmet' in class_name or 'with' in class_name:
            helmet_count += 1
        elif 'without' in class_name or 'no' in class_name:
            no_helmet_count += 1
        elif 'person' in class_name:
            person_count += 1
        elif 'motor' in class_name or 'bike' in class_name:
            bike_count += 1

    print("\n" + "="*70)
    print("📊 DETECTION SUMMARY")
    print("="*70)
    print(f"✅ With Helmet:    {helmet_count} detected")
    print(f"❌ Without Helmet: {no_helmet_count} detected")
    print(f"👤 Persons:        {person_count} detected")
    print(f"🏍️  Motorcycles:    {bike_count} detected")
    print("="*70)

    if no_helmet_count > 0:
        print("⚠️  WARNING: Riders without helmets detected!")
    elif helmet_count > 0:
        print("✅ All riders are wearing helmets!")
    print("="*70 + "\n")

print("✅ Detection functions ready!\n")

# STEP 6: Choose Detection Mode
# ============================================================================
print("="*70)
print("🎯 STEP 6/6: READY TO DETECT HELMETS!")
print("="*70)
print("\n📋 Choose detection mode:\n")
print("1️⃣  Upload IMAGE file (JPG, PNG)")
print("2️⃣  Upload VIDEO file (MP4, AVI)")
print("3️⃣  Use WEBCAM (real-time)")
print("4️⃣  Test with DEMO image")
print("="*70)

choice = input("\n👉 Enter your choice (1/2/3/4): ").strip()

# ============================================================================
# OPTION 1: IMAGE DETECTION
# ============================================================================
if choice == '1':
    print("\n📤 Upload your image file...")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\n🔍 Analyzing: {filename}")
        print("⏳ Processing...\n")

        # Detect
        results = model(filename)

        # Show detailed results
        print("📋 DETAILED DETECTIONS:")
        print(results.pandas().xyxy[0].to_string())

        # Analyze
        analyze_results(results)

        # Display image with detections
        img = cv2.imread(filename)
        img = draw_custom_boxes(img, results)

        print("🖼️  RESULT IMAGE:")
        cv2_imshow(img)

        # Save result
        output = f'detected_{filename}'
        cv2.imwrite(output, img)
        print(f"\n💾 Saved as: {output}")

# ============================================================================
# OPTION 2: VIDEO DETECTION
# ============================================================================
elif choice == '2':
    print("\n📤 Upload your video file...")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\n🎥 Processing video: {filename}")
        print("⏳ This may take several minutes...\n")

        cap = cv2.VideoCapture(filename)

        # Video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"📹 Video: {width}x{height}, {fps}fps, {total_frames} frames\n")

        # Output video
        output_file = f'detected_{filename}'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

        frame_num = 0
        total_helmet = 0
        total_no_helmet = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Detect in frame
            results = model(frame)

            # Count detections
            detections = results.pandas().xyxy[0]
            for idx, row in detections.iterrows():
                if 'helmet' in row['name'].lower() or 'with' in row['name'].lower():
                    total_helmet += 1
                elif 'without' in row['name'].lower() or 'no' in row['name'].lower():
                    total_no_helmet += 1

            # Draw boxes
            frame = draw_custom_boxes(frame, results)
            out.write(frame)

            frame_num += 1

            if frame_num % 30 == 0:
                progress = (frame_num / total_frames) * 100
                print(f"⏳ Progress: {frame_num}/{total_frames} ({progress:.1f}%) | "
                      f"Helmets: {total_helmet} | No Helmets: {total_no_helmet}")

        cap.release()
        out.release()

        print(f"\n✅ Video processing complete!")
        print(f"💾 Output saved as: {output_file}")
        print(f"\n📊 Total detections in video:")
        print(f"   ✅ With Helmet: {total_helmet}")
        print(f"   ❌ Without Helmet: {total_no_helmet}")

# ============================================================================
# OPTION 3: WEBCAM DETECTION
# ============================================================================
elif choice == '3':
    print("\n📸 Activating webcam...")

    from IPython.display import Javascript
    from google.colab.output import eval_js
    from base64 import b64decode

    def capture_photo(filename='webcam.jpg'):
        js = Javascript('''
            async function takePhoto() {
                const div = document.createElement('div');
                div.style.textAlign = 'center';

                const capture = document.createElement('button');
                capture.textContent = '📸 CAPTURE PHOTO';
                capture.style.padding = '15px 30px';
                capture.style.fontSize = '18px';
                capture.style.backgroundColor = '#4CAF50';
                capture.style.color = 'white';
                capture.style.border = 'none';
                capture.style.borderRadius = '5px';
                capture.style.cursor = 'pointer';
                capture.style.marginTop = '10px';

                const video = document.createElement('video');
                video.style.display = 'block';
                video.style.margin = '0 auto';
                video.style.maxWidth = '640px';
                video.style.border = '3px solid #4CAF50';

                const stream = await navigator.mediaDevices.getUserMedia({video: true});

                document.body.appendChild(div);
                div.appendChild(video);
                div.appendChild(capture);
                video.srcObject = stream;
                await video.play();

                google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

                await new Promise((resolve) => capture.onclick = resolve);

                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                canvas.getContext('2d').drawImage(video, 0, 0);
                stream.getVideoTracks()[0].stop();
                div.remove();
                return canvas.toDataURL('image/jpeg', 0.9);
            }
            ''')
        display(js)
        data = eval_js('takePhoto()')
        binary = b64decode(data.split(',')[1])
        with open(filename, 'wb') as f:
            f.write(binary)
        return filename

    photo = capture_photo('webcam_capture.jpg')

    print("\n🔍 Detecting helmets in captured photo...")

    results = model(photo)

    print("📋 DETAILED DETECTIONS:")
    print(results.pandas().xyxy[0].to_string())

    analyze_results(results)

    img = cv2.imread(photo)
    img = draw_custom_boxes(img, results)

    print("🖼️  RESULT:")
    cv2_imshow(img)

# ============================================================================
# OPTION 4: DEMO TEST
# ============================================================================
elif choice == '4':
    print("\n🌐 Downloading demo image...")

    # Download a sample bike image
    demo_url = "https://images.unsplash.com/photo-1558981806-ec527fa84c39?w=800"
    urllib.request.urlretrieve(demo_url, "demo.jpg")

    print("🔍 Running helmet detection on demo image...")

    results = model('demo.jpg')

    print("\n📋 DETAILED DETECTIONS:")
    print(results.pandas().xyxy[0].to_string())

    analyze_results(results)

    img = cv2.imread('demo.jpg')
    img = draw_custom_boxes(img, results)

    print("🖼️  RESULT:")
    cv2_imshow(img)

else:
    print("\n❌ Invalid choice! Run again and choose 1, 2, 3, or 4")

# ============================================================================
# FINAL NOTES
# ============================================================================
print("\n" + "="*70)
print("🎉 HELMET DETECTION COMPLETE!")
print("="*70)
print("""
💡 TIPS:
- Green boxes = Person WITH helmet ✅
- Red boxes = Person WITHOUT helmet ❌
- Blue boxes = Other objects (bike, person, etc.)

📊 Model detects:
- Riders wearing helmets
- Riders NOT wearing helmets
- Motorcycles
- Persons

🔧 To improve accuracy:
- Use good quality images/videos
- Ensure proper lighting
- Camera should clearly see the head area

💾 All results are saved in the current directory!
""")
print("="*70)

⏳ STEP 1/6: Installing YOLOv5... (2-3 minutes)
✅ YOLOv5 installed!

⏳ STEP 2/6: Loading libraries...
✅ Libraries loaded!

⏳ STEP 3/6: Downloading CUSTOM helmet detection model...
📥 This model is specifically trained to detect helmets!

   Trying source 1...
   ⚠️  Source 1 unavailable, trying next...
   Trying source 2...
   ⚠️  Source 2 unavailable, trying next...

📦 Automatic download failed. Using alternative method...
🔄 Installing from Roboflow Universe...
⚠️  Roboflow method failed too.

📤 MANUAL UPLOAD REQUIRED
Please download a helmet detection model manually:

1. Visit: https://universe.roboflow.com/search?q=helmet+detection
2. Choose a 'YOLOv5' format model
3. Download the 'best.pt' file
4. Upload it below when prompted

